# EBM Internals - Multiclass

This is part 3 of a 3 part series describing EBM internals and how to make predictions. For part 1, click [here](./ebm-internals-regression.ipynb). For part 2, click [here](./ebm-internals-classification.ipynb).

In this part 3 we'll cover multiclass, specified bin cuts, term exclusion, and unknown values. Before reading this part you should be familiar with the information in [part 1](./ebm-internals-regression.ipynb) and [part 2](./ebm-internals-classification.ipynb)

In [ ]:
# boilerplate
from interpret import show
from interpret.glassbox import ExplainableBoostingClassifier
import numpy as np

from interpret import set_visualize_provider
from interpret.provider import InlineProvider
set_visualize_provider(InlineProvider())

In [ ]:
# make a dataset composed of a nominal, an unused feature, and a continuous 
X = [["Peru", "", 7], ["Fiji", "", 8], ["Peru", "", 9], [None, "", None]]
y = [6000, 5000, 4000, 6000] # integer classes

# Fit a classification EBM without interactions
# Specify exact bin cuts for the continuous feature
# Exclude the middle feature during fitting
# Eliminate the validation set to handle the small dataset
ebm = ExplainableBoostingClassifier(
    interactions=0, 
    feature_types=['nominal', 'nominal', [7.25, 9.0]], 
    exclude=[(1,)],
    validation_size=0, outer_bags=1, min_samples_leaf=1)
ebm.fit(X, y)
show(ebm.explain_global())

<br/>
<br/>
<br/>
<br/>
<br/>


In [ ]:
print(ebm.classes_)

Like all scikit-learn classifiers, we store the list of classes in the ebm.classes_ attribute as a sorted array. In this example our classes are integers, but we also accept strings as seen in [part 2](./ebm-internals-classification.ipynb)

In [ ]:
print(ebm.feature_types)

In this example we passed feature_types into the \_\_init\_\_ function of the ExplainableBoostingClassifier. Per scikit-learn convention, this was recorded unmodified in the ebm object.

In [ ]:
print(ebm.feature_types_in_)

The feature_types passed into \_\_init\_\_ were actualized into the base feature types of ['nominal', 'nominal', 'continuous'].

In [ ]:
print(ebm.feature_names)

feature_names were not specified in the call to the \_\_init\_\_ function of the ExplainableBoostingClassifier, so it was set to None following the scikit-learn convention of recording \_\_init\_\_ parameters unmodified.

In [ ]:
print(ebm.feature_names_in_)

Since we did not specify feature names, some default names were created for the model. If we had passed feature_names to the \_\_init\_\_ function of the ExplainableBoostingClassifier, or if we had used a Pandas dataframe with column names, then ebm.feature_names_in_ would have contained those names.  Following scikit-learn's [SLEP007 convention](https://scikit-learn-enhancement-proposals.readthedocs.io/en/latest/slep007/proposal.html), we recorded this in ebm.feature_names_in_

In [ ]:
print(ebm.term_features_)

In the call to the \_\_init\_\_ function of the ExplainableBoostingClassifier, we specified exclude=[(1,)], which means we excluded the middle feature in the list of terms for the model. The middle feature is thus missing from the list of terms in ebm.term_features_

In [ ]:
print(ebm.term_names_)

ebm.term_names_ is also missing the middle feature since ebm.term_features_ is missing that feature 

In [ ]:
print(ebm.bins_)

ebm.bins_ is a per-feature attribute, so the middle feature is listed here. We see however that the middle feature does not have a binning definition since it is not considered when making predictions with the model.

These bins are structured as described in [part 1](./ebm-internals-regression.ipynb) and [part 2](./ebm-internals-classification.ipynb). One change to note though is that the continuous feature bin cuts are the same as the bin cuts [7.25, 9.0] specified in the feature_types parameter to the \_\_init\_\_ function of the ExplainableBoostingClassifier.

It is also noteworthy that the last bin cut specified is exactly equal to the largest feature value of 9.0. In this situation where a feature value is identical to the cut value, the feature gets placed into the upper bin.

In [ ]:
print(ebm.intercept_)

For multiclass, ebm.intercept_ is an array containing a logit for each of the predicted classes in ebm.classes_. This behavior is identical to how other scikit-learn multiclass classifiers generate one logit per class.

In [ ]:
print(ebm.term_scores_[0])

ebm.term_scores_[0] is the lookup table for the nominal categorical feature containing country names. For multiclass, each bin consists of an array of logits with 1 logit per class being predicted. In this example, each row corresponds to a bin. There are 4 bins in the outer index and 3 class logits in the inner index.

Missing values are once again placed in the 0th bin index, shown above as the first row of 3 logits. The unknown bin is the last row of zeros.

Since this feature is a nominal categorial, we use the dictionary {'Fiji': 1, 'Peru': 2} to lookup which row of logits to use for each categorical string.

In [ ]:
print(ebm.term_scores_[1])

ebm.term_scores_[1] is the lookup table for the continuous feature. Once again, the 0th and last index are for missing values, and unknown values respectively. This particular example has 5 bins consisting of the 0th missing bin index, the three partitions from the 2 cuts, and the unknown bin index. Each row is a single bin that contains 3 class logits. 

# Sample code

This sample code incorporates everything discussed in all 3 sections. It could be used as a drop in replacement for the existing EBM predict function of the ExplainableBoostingRegressor or as the predict_proba function of the ExplainableBoostingClassifier.

In [ ]:
from sklearn.utils.extmath import softmax

sample_scores = []
for sample in X:
    # start from the intercept for each sample
    score = ebm.intercept_.copy()
    if isinstance(score, float) or len(score) == 1:
        # regression or binary classification
        score = float(score)

    # we have 2 terms, so add their score contributions
    for term_idx, features in enumerate(ebm.term_features_):
        # indexing into a tensor requires a multi-dimensional index
        tensor_index = []

        # main effects will have 1 feature, and pairs will have 2 features
        for feature_idx in features:
            feature_val = sample[feature_idx]
            bin_idx = 0  # if missing value, use bin index 0

            if feature_val is not None and feature_val is not np.nan:
                # we bin differently for main effects and pairs, so first 
                # get the list containing the bins for different resolutions
                bin_levels = ebm.bins_[feature_idx]

                # what resolution do we need for this term (main resolution, pair
                # resolution, etc.), but limit to the last resolution available
                bins = bin_levels[min(len(bin_levels), len(features)) - 1]

                if isinstance(bins, dict):
                    # categorical feature
                    # 'unknown' category strings are in the last bin (-1)
                    bin_idx = bins.get(feature_val, -1)
                else:
                    # continuous feature
                    try:
                        # try converting to a float, if that fails it's 'unknown'
                        feature_val = float(feature_val)
                        # add 1 because the 0th bin is reserved for 'missing'
                        bin_idx = np.digitize(feature_val, bins) + 1
                    except ValueError:
                        # non-floats are 'unknown', which is in the last bin (-1)
                        bin_idx = -1
        
            tensor_index.append(bin_idx)
        # local_score is also the local feature importance
        local_score = ebm.term_scores_[term_idx][tuple(tensor_index)]
        score += local_score
    sample_scores.append(score)

predictions = np.array(sample_scores)

if hasattr(ebm, 'classes_'):
    # classification
    if len(ebm.classes_) == 2:
        # binary classification

        # softmax expects two logits for binary classification
        # the first logit is always equivalent to 0 for binary classification
        predictions = [[0, x] for x in predictions]
    predictions = softmax(predictions)

if hasattr(ebm, 'classes_'):
    print("probabilities for classes " + str(ebm.classes_))
    print("")
    print(ebm.predict_proba(X))
else:
    print(ebm.predict(X))
print("")
print(predictions)